Prompt Depth Anything
====

**Prompting Depth Anything for 4K Resolution Accurate Metric Depth Estimation**

 * Paper: https://arxiv.org/abs/2412.14015

![Prompt Depth Anything](../assets/promptdepthanything_overview.png)